## Sentiment Analysis Tool

In [16]:
# Using Textblob to analyze sentiment example
from textblob import TextBlob

text = "I THINK dodgecoin is great! I'm so excited to buy more!"
blob = TextBlob(text)
sentiment = blob.sentiment

print(f"Polarity: {sentiment.polarity}, Subjectivity: {sentiment.subjectivity}")


# Polarity is a float within the range [-1.0, 1.0], where -1 means negative sentiment and 1 means positive sentiment.
# Subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.


Polarity: 0.6666666666666666, Subjectivity: 0.6666666666666666


## Twitter API

In [17]:
import requests
import os
from dotenv import load_dotenv


load_dotenv()
# Now you can access your environment variable securely
bearer_token = os.getenv('TWITTER_BEARER_TOKEN')



def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def create_url():
    hashtag = "%23crypto"
    exclude_phrases = "-\"signing up\" -\"receive tokens\" -giveaway"
    additional_keywords = "\"crypto\" OR \"cryptocurrency\""
    language = "lang:en"
    # Ensure proper spacing and logical grouping
    query = f"{hashtag} {exclude_phrases} {additional_keywords} {language}"
    # Forming the URL
    url = f"https://api.twitter.com/2/tweets/search/recent?query={query}&tweet.fields=author_id,created_at"
    return url


def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()



In [18]:
# Set up the URL and headers
url = create_url()
headers = create_headers(bearer_token)

# Make the request
json_response = connect_to_endpoint(url, headers)

# Print the response
print(json_response)


{'data': [{'edit_history_tweet_ids': ['1765974009533894865'], 'created_at': '2024-03-08T05:33:08.000Z', 'id': '1765974009533894865', 'text': 'RT @TeaSwap_org: TeaSwap will revolutionize the way you buy cryptocurrencies, making it safer, easier, and cheaper.\n\nWe want your input: wh…', 'author_id': '1761998509480263680'}, {'edit_history_tweet_ids': ['1765974007789019458'], 'created_at': '2024-03-08T05:33:08.000Z', 'id': '1765974007789019458', 'text': 'Like , Comment and Rt \nif you love Dogemars $MARS 🐕💎❤️\n\n#Binance #Crypto #memecoin \n$ETH $BTC $BONK $PEPE $SHIB \n$WIF $SOL https://t.co/PWJFcB3buy', 'author_id': '1731668982569086976'}, {'edit_history_tweet_ids': ['1765974007495426189'], 'created_at': '2024-03-08T05:33:08.000Z', 'id': '1765974007495426189', 'text': "RT @Vicki_Swift_: Telos @HelloTelos impresses with Reactor Fusion's RF Staker Rewards Event! 💰 Stakers on #TelosEVM have a golden opportuni…", 'author_id': '1722634657001558016'}, {'edit_history_tweet_ids': ['176597400710

In [19]:
# storing tweet
import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('crypto_tweets.db')
c = conn.cursor()

# Create a table to store tweets
c.execute('''CREATE TABLE IF NOT EXISTS tweets
             (id TEXT PRIMARY KEY, author_id TEXT, created_at TEXT, text TEXT)''')

conn.commit()



In [20]:
def insert_tweet_data(json_response):
    tweets = json_response.get('data', [])
    for tweet in tweets:
        c.execute('INSERT OR IGNORE INTO tweets (id, author_id, created_at, text) VALUES (?, ?, ?, ?)',
                  (tweet['id'], tweet['author_id'], tweet['created_at'], tweet['text']))

    conn.commit()

insert_tweet_data(json_response)


In [10]:
c.execute('SELECT * FROM tweets')
for row in c.fetchall():
    print(row)


('1758835781643407742', '1758101917669167104', '2024-02-17T12:48:22.000Z', 'RT @Mazimatic: #Bitcoin is soaring high again, breaking the $50K barrier! 🚀\n\nAre you ready for the next big prediction? Keep your eyes on t…')
('1758835780829536596', '1063709691484618752', '2024-02-17T12:48:22.000Z', '@bucket_protocol @cedelabs Check out this amazing project building the future of Cryptocurrency and Blockchain technology @tzibalone27 @BaedowiZacky  @LayaneThais5\n\n0x0f521f35566646fcd47329ceb01e11105f46305d109255236aafad0588398d56')
('1758835780095726043', '1601513746341371906', '2024-02-17T12:48:22.000Z', "RT @HpXentity: $hpxt #airdrop is now live!\n\nWe've made it as simple as possible for you to receive your airdrop tokens! Just fill in and fo…")
('1758835780024226212', '1758169816106463232', '2024-02-17T12:48:22.000Z', 'RT @Jessica194102: Get ready to step into the future of cryptocurrency with our captivating  3D art depicting the @Persona_Journey  project…')
('1758835778648408450', '77

In [21]:
c.execute('SELECT COUNT (*) FROM tweets')
print(c.fetchone()[0])

80


In [22]:
import pandas as pd
import sqlite3

# Connect to your SQLite database
conn = sqlite3.connect('crypto_tweets.db')

# SQL query to select all tweets
query = "SELECT * FROM tweets"

# Load the results of the query into a DataFrame
tweets_df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Now you can work with the tweets_df DataFrame
print(tweets_df.head())


                    id            author_id                created_at  \
0  1758835781643407742  1758101917669167104  2024-02-17T12:48:22.000Z   
1  1758835780829536596  1063709691484618752  2024-02-17T12:48:22.000Z   
2  1758835780095726043  1601513746341371906  2024-02-17T12:48:22.000Z   
3  1758835780024226212  1758169816106463232  2024-02-17T12:48:22.000Z   
4  1758835778648408450            773288940  2024-02-17T12:48:21.000Z   

                                                text  
0  RT @Mazimatic: #Bitcoin is soaring high again,...  
1  @bucket_protocol @cedelabs Check out this amaz...  
2  RT @HpXentity: $hpxt #airdrop is now live!\n\n...  
3  RT @Jessica194102: Get ready to step into the ...  
4  @ice_blockchain ✅ Verifying my account on @ice...  


In [23]:
# Example: Count tweets by author_id
tweet_counts = tweets_df['author_id'].value_counts()
print(tweet_counts.head())

# Example: Simple sentiment analysis (assuming you have a 'sentiment' column)
# You might need to calculate this column beforehand
if 'sentiment' in tweets_df.columns:
    average_sentiment = tweets_df['sentiment'].mean()
    print(f"Average Sentiment: {average_sentiment}")


author_id
1723222904408571905    3
1545244142359412736    2
1606190075599097856    2
1042388521             2
1707854110425550848    1
Name: count, dtype: int64


In [24]:
tweets_df['text'] = tweets_df['text'].astype(str)
print(tweets_df['text'][50])

RT @stratisplatform: Announcing Stratis Auroria testnet farming campaign. 500,000 STRAX EVM to airdrop!

Join the airdrop now⬇️

Info: http…


In [25]:
from textblob import TextBlob


tweet = tweets_df['text'].iloc[-1]
print(tweet)

tweet_sentiment = TextBlob(tweet).sentiment
print(tweet_sentiment)

@samal2052 Wow, the environmental impact of cryptocurrency mining is concerning. I hope more sustainable solutions are implemented soon.
Sentiment(polarity=0.3, subjectivity=0.75)


In [ ]:
import praw
from dotenv import load_dotenv
import os

## Reddit API

In [ ]:
import praw
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access environment variables
client_id = os.getenv('REDDIT_CLIENT_ID')
client_secret = os.getenv('REDDIT_CLIENT_SECRET')
user_agent = os.getenv('REDDIT_USER_AGENT')
username = os.getenv('REDDIT_USERNAME')
password = os.getenv('REDDIT_PASSWORD')



# Initialize PRAW with your credentials
reddit = praw.Reddit(client_id=client_id,
                     client_secret=	client_secret,
                     user_agent=user_agent,
                     username= username,
                     password= password)
print('Done')


In [ ]:
subreddit = reddit.subreddit("crypto")

for post in subreddit.top(limit=10):
    print(post.title, post.url)
